# T-S diagrams from particle tracks

## part of a project to quantify transports across the Greenland-Scotland section by source and destination

- Lagrangian particle tracking in VIKING20X model

- uses EOS-80 equation of state (seawater package) as used in Viking20x 

## technical preamble

In [ ]:
%matplotlib inline

In [ ]:


from pathlib import Path
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
import seaborn as sns
import cmocean as co
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seawater as sw
np.warnings.filterwarnings('ignore')


## parameters

In [ ]:
# release time

trelease = np.datetime64('2015-12-16T12:00')

# Project path
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

# project_path = Path("C:/Users/sa05af/OneDrive - SAMS/Projects/parcels_quantitative_202004/")

# select section FS, IF, or GI

section = 'FS'

if section == 'FS':
    
    # Faroe-Shetland
    path_data_tracks = Path('data/processed/tracks/FaroeShetland/') 
    filename = 'test_salinity_FS.nc'
#     filenameb = 'tracks_parcels_FS_backwards_201512.nc'

    #velocity conversions on FS channel (61N)

    lat2metres = 60.0*1852.0
    lon2metres = 60.0*1852.0*np.cos(np.radians(61.0))

    sealandratio = 5591.0/20000.0
    sectionAreaEW = 5.7 * 1500.0 * sealandratio * lon2metres  # very approx. mean depth x length between slopes.
    sectionAreaNS = 1.87 * 1500.0 * sealandratio * lat2metres

    
# elif section == 'IF':
    
#     # Iceland-Faroe
#     path_data_tracks = Path('data/processed/tracks/IcelandFaroe/') 
#     filename = 'tracks_parcels_IFR_forwards_201512.nc'
#     filenameb = 'tracks_parcels_IFR_backwards_201512.nc'

#     #velocity conversions on IFR (63.5N)

#     lat2metres = 60.0*1852.0
#     lon2metres = 60.0*1852.0*np.cos(np.radians(63.5))

#     sealandratio = 9308.0/20000.0
#     sectionAreaEW = 7.0 * 700.0 * sealandratio * lon2metres  # very approx. mean depth x length between slopes.
#     sectionAreaNS = 2.8 * 700.0 * sealandratio * lat2metres
    
# else:

#     # Greenland-Iceland
#     path_data_tracks = Path('data/processed/tracks/GreenlandIceland/') 
#     filename = 'tracks_parcels_GI_forwards_201512.nc'
#     filenameb = 'tracks_parcels_GI_backwards_201512.nc'

#     #velocity conversions on GI (67.3N)
#     lat2metres = 60.0*1852.0
#     lon2metres = 60.0*1852.0*np.cos(np.radians(67.3))

#     sealandratio = 2420.0/10000.0
#     sectionAreaEW = 5.73 * 1500.0 * sealandratio * lon2metres  # very approx. mean depth x length between slopes.
#     sectionAreaNS = 2.94 * 1500.0 * sealandratio * lat2metres

# model mask file
data_path = Path("data/external/iAtlantic/")
experiment_name = "VIKING20X.L46-KKG36107B"
mesh_mask_file = project_path / data_path / "mask" / experiment_name / "1_mesh_mask.nc"


## helper functions

In [ ]:
def transports(dsb, nParticles):
    transport = (sectionAreaEW*dsb.isel(obs=0).vvel*lat2metres +
                            sectionAreaNS*dsb.isel(obs=0).uvel*lon2metres)/nParticles
    speed = np.sqrt((dsb.isel(obs=0).vvel*lat2metres)**2 + (dsb.isel(obs=0).uvel*lon2metres)**2)
    return transport,speed

### forward tracks then backward tracks

In [ ]:
ds = xr.open_dataset(project_path / path_data_tracks / filename)
ds.temp.attrs['standard_name'] = 'potential temperature'
ds.temp.attrs['units'] = 'degree_C'
ds.salt.attrs['standard_name'] = 'practical salinity'
ds.salt.attrs['units'] = 'PSU'
# dsb = xr.open_dataset(project_path / path_data_tracks / filenameb)
# dsb.temp.attrs['standard_name'] = 'temperature'
# dsb.temp.attrs['units'] = 'degree_C'

# ds_full = xr.concat([dsb,ds],dim='obs')

display(ds)


## Calculate some basic parameters from data for later

In [ ]:
nParticles = ds.sizes['traj']
nobsb = ds.sizes['obs']

# calculate initial transports

particle_transport,speed0 = transports(ds,nParticles)
sizes = np.abs(particle_transport)/400.0


## calculate potential densities

In [ ]:
# there are neater ways to do this with xarray if the datasets are large

S = ds.where(ds.salt != 0.0).salt.data.flatten()
T = ds.where(ds.salt != 0.0).temp.data.flatten()
time = ds.where(ds.salt != 0.0).time.data.flatten()
z = ds.where(ds.salt != 0.0).z.data.flatten()
sig0 = sw.pden(S,T,0.0,0.0)
sig0 -= 1000.0

T_pad = (np.nanmax(T) - np.nanmin(T))/20.0
S_pad = (np.nanmax(S) - np.nanmin(S))/20.0
T_lim = [np.nanmin(T) - T_pad, np.nanmax(T) + T_pad]
S_lim = [np.nanmin(S) - S_pad, np.nanmax(S) + S_pad]


## contruct T-S plot

In [ ]:
def TSplot_colourbytime(S,T,time,S_lim,T_lim):
    Tgrid = np.zeros((100,100)) + np.linspace(T_lim[0],T_lim[1],100)
    Sgrid = np.zeros((100,100)) + np.linspace(S_lim[0],S_lim[1],100)
    sig0grid = sw.pden(Sgrid,Tgrid.T,0,0) - 1000.0

    fig,ax = plt.subplots(figsize=(8,8))

    cs = ax.contour(Sgrid,Tgrid.T,sig0grid,
              colors = 'grey', linestyles = 'dashed')

    ax.clabel(cs, inline=1, fmt='%1.1f', fontsize=10)

    cm = ax.scatter(S, T, 
                    s = 2, 
                    c = mdates.date2num(time),
                    cmap = co.cm.matter,
                    zorder=10,
                    alpha=0.3)

    cbar = fig.colorbar(cm)
    # set alpha to 1 on colorbar
    cbar.set_alpha(1.0)
    cbar.draw_all() # don't understand why this is required but alph doesn't change without it
    # format the dates on colorbar
    loc = mdates.AutoDateLocator()
    cbar.ax.yaxis.set_major_locator(loc)
    cbar.ax.yaxis.set_major_formatter(mdates.ConciseDateFormatter(loc))
    # label axes
    ax.set_ylabel('potential temperature [$\degree$C]')
    ax.set_xlabel('practical salinity [PSU]')

In [ ]:
def TSplot(S,T,z,zlabel,S_lim,T_lim):
    Tgrid = np.zeros((100,100)) + np.linspace(T_lim[0],T_lim[1],100)
    Sgrid = np.zeros((100,100)) + np.linspace(S_lim[0],S_lim[1],100)
    sig0grid = sw.pden(Sgrid,Tgrid.T,0,0) - 1000.0

    fig,ax = plt.subplots(figsize=(8,8))

    cs = ax.contour(Sgrid,Tgrid.T,sig0grid,
              colors = 'grey', linestyles = 'dashed')

    ax.clabel(cs, inline=1, fmt='%1.1f', fontsize=10)

    cm = ax.scatter(S, T, 
                    s = 2, 
                    c = z,
                    cmap = co.cm.matter,
                    zorder=10,
                    alpha=0.3)

    cbar = fig.colorbar(cm,label = zlabel)
    # set alpha to 1 on colorbar
    cbar.set_alpha(1.0)
    cbar.draw_all() # don't understand why this is required but alph doesn't change without it
    # label axes
    ax.set_ylabel('potential temperature [$\degree$C]')
    ax.set_xlabel('practical salinity [PSU]')

In [ ]:
TSplot_colourbytime(S,T,time,S_lim,T_lim)

In [ ]:
TSplot(S,T,z,'depth [m]',S_lim,T_lim)